In [1]:
# Data manipulation
import pandas as pd
import numpy as np
import pendulum as plm

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

In [2]:
# 1 groupby

df = pd.DataFrame({'col1': ['apple', 'banana', 'orange'] * 30,
                   'col2': np.random.rand(90),
                   'col3': np.random.randint(0, 15, 90)})

# groupby.agg(new_col_name = ('colname',agg_func on pd.Series))

(
    df
    .groupby('col1',as_index=False)
    .agg(
        sum_col2 = ('col2',np.sum), # perform aggregation
        distict_col2 = ('col2',lambda x: x.nunique()),
        collect_list_col3 = ('col3',lambda x : x.tolist()),
        collect_list_sampled_5_col3 = ('col3',lambda x : x.sample(5,random_state=42).tolist())
    )
)

,col1,sum_col2,distict_col2,collect_list_col3,collect_list_sampled_5_col3
0,apple,16.534381,30.0,"[2, 2, 0, 1, 6, 9, 12, 9, 9, 10, 3, 8, 6, 11, ...","[3, 11, 12, 14, 9]"
1,banana,13.328910,30.0,"[11, 5, 12, 0, 11, 12, 12, 13, 7, 10, 12, 0, 1...","[1, 3, 11, 1, 7]"
2,orange,16.422880,30.0,"[7, 11, 12, 1, 2, 6, 13, 3, 10, 9, 12, 0, 0, 7...","[8, 14, 4, 13, 10]"


In [3]:
# case when pattern

df = pd.DataFrame({'col1': ['apple', 'banana', 'orange'] * 30,
                   'col2': np.random.rand(90),
                   'col3': np.random.randint(0, 15, 90)})

df = (
    df
    .assign(
        col4 = (
           np.where(df['col2'] <= 5, '0_5',
           np.where(df['col2'] <= 15,'6_15',
           np.where(df['col2'] <= 25,'16_25',
           np.where(df['col2'] <= 50,'25_50',
           np.where(df['col2'] <= 100,'50_100',
            '100_'
                    )))))))
)

df.head()

,col1,col2,col3,col4
0,apple,0.881429,4,0_5
1,banana,0.974353,4,0_5
2,orange,0.985837,9,0_5
3,apple,0.824506,9,0_5
4,banana,0.581750,5,0_5


In [22]:
# where pattern


def create_datetime(start = plm.DateTime(2023,1,1),ndays = 90):
    start = start
    ndays = 90
    end = start.add(ndays)
    return [dt for dt in plm.period(start,end).range('days')]



df = pd.DataFrame({'col1': ['apple', 'banana', 'orange'] * 30,
                   'col2': np.random.rand(90),
                   'col3': np.random.randint(0, 15, 90),
                   'time_col' : create_datetime()[:90]
                  })

df.info()

(
    df
    .where(
    pd.to_datetime(df['time_col'],infer_datetime_format=True) < '2023-01-02'
    )
    .dropna(subset=['time_col'])
)
                  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   col1      90 non-null     object        
 1   col2      90 non-null     float64       
 2   col3      90 non-null     int64         
 3   time_col  90 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 2.9+ KB


,col1,col2,col3,time_col
0,apple,0.280618,9.0,2023-01-01
